# Discrete Element Method for Rigid Body Simulation

For this project, I have implemented a method of simulating rigid body dynamics using the discrete element approximation, which treats each rigid body as being composed of discrete particles. While this method has some drawbacks such as dependence on the sampling resolution of the particles for the quality of results, it is extremely well suited for parallel implementation on the GPU. It is also agnostic to model topology, unlike some analytic methods which work best on convex models, for instance.

## Overview

The pipeline is divided into the following stages:
- Sample points in the volume of each mesh with uniform density
- Precompute various invariant properties of each rigid body, such as the mass and moment of inertia tensor
- Allocate a grid data structure to keep track of the spatial locations of each object's particles
- For each time step:
 - Repopulate the data structure using a CUDA scattering kernel
 - Compute the forces on each particle, along with resulting torques at each point, by searching the neighboring grid cells for nearby particles (also in a CUDA kernel)
 - Using CUDA reduction, compute the net force and torque on each rigid body as a whole
 - Update the state by integrating the laws of motion
 
## Inter-particle forces
By the rigid body assumption, the net force due to other particles in the same object is zero. So for all particles in different objects, we define the following forces:

$$\mathbf f_{i,s}=-k(d-|\mathbf r_{ij}|)\frac{\mathbf r_{ij}}{|\mathbf r_{ij}|}$$

is the spring force that acts when particles $i$ and $j$ are less than distance $d$ apart, and

$$\mathbf f_{i,d}=\eta \mathbf v_{ij}$$

is the damping force (where $\mathbf v_{ij} is the relative velocity of the two particles).

### Updating state
We use the laws of motion for rigid body mechanics. Angular momentum and momentum are affected by torque and force, respectively:
$$\dot L = \mathbf \tau$$
$$\dot p = \mathbf F$$
where $\tau=\sum_i (r_i-c)\times F_i$ and $c$ is the center of mass.
Velocity is related to momentum by the simple formula $v=p/M$, but rotations are somewhat trickier. Angular velocity $\omega$ is related to angular momentum via the *inertia tensor* $I$, so that $$L = I\omega$$. $I$ also varies in time, but it turns out that a time-invariant matrix can be precomputed in the rigid body's rest coordinate frame. For more info on the details, see https://www.cs.cmu.edu/~baraff/sigcourse/notesd1.pdf.


## Viewing results
For now, the closest thing to displaying 3D animations in these notebooks that I have found is to have interactive plots, with a slider controlling the frame under examination.

In [1]:
from meshplot import plot, interact
import ipywidgets
import numpy as np
import numpy.linalg as la
import igl
import time, math

In [2]:
from numba import cuda, jit, prange, njit, int32, float32, void

## Utility functions for precomputing samples, and integrating mass and moment of inertia

In [3]:
def voxel_points(axes):
    """generate n-dimensional grid coordinates from the given set of (x, y, z...) values"""
    ndims = len(axes)
    dims = [len(x) for x in axes]
    indices = np.indices(dims)
    points = np.array([axis[index] for axis, index in zip(axes, indices)])
    points = points.reshape([ndims, np.prod(dims)])
    return points.T.copy()

In [4]:
def sample_mesh_interior(V, F, gridLength, dtype=np.float32):
    minPt = np.min(V, 0)
    maxPt = np.max(V, 0)
    q = voxel_points((np.arange(minPt[0], maxPt[0] + gridLength, gridLength, dtype=dtype),
                       np.arange(minPt[1], maxPt[1] + gridLength, gridLength, dtype=dtype),
                         np.arange(minPt[2], maxPt[2] + gridLength, gridLength, dtype=dtype)))
    #print(q)
    w = igl.fast_winding_number_for_meshes(V.astype(dtype), F, q)
    #print(w.shape)
    return q[w > 0.5]

@njit (parallel=True)
def compute_inertia_tensor(points, pointmass):
    inertia_tensor = np.zeros((3, 3), dtype=points.dtype)
    N = points.shape[0]
    for i in prange(N):
        pointCM = points[i,:]
        moments = pointCM * pointCM
        inertia_tensor[0, 0] += moments[1] + moments[2]
        inertia_tensor[1, 1] += moments[0] + moments[2]
        inertia_tensor[2, 2] += moments[0] + moments[1]
        inertia_tensor[0, 1] = pointCM[0] * pointCM[1]
        inertia_tensor[0, 2] = pointCM[0] * pointCM[2]
        inertia_tensor[1, 2] = pointCM[1] * pointCM[2]
    inertia_tensor *= pointmass
    inertia_tensor[1, 0] = inertia_tensor[0, 1]
    inertia_tensor[2, 0] = inertia_tensor[0, 2]
    inertia_tensor[2, 1] = inertia_tensor[1, 2]
    return inertia_tensor

In [5]:
class PhysObject:
    def __init__(self, V, F, gridDim, density=1):
        self.points = sample_mesh_interior(V, F, gridDim)
        pointmass = density * gridDim ** 3
        cm = np.mean(self.points, 0)
        self.points -= cm
        self.mass = pointmass * self.points.shape[0]
        self.inertia_tensor = compute_inertia_tensor(self.points, pointmass)
        self.inertia_tensor_inv = la.inv(self.inertia_tensor)

gridDim = 0.02
start = time.time()
physobj = PhysObject(v, f, gridDim)
end = time.time()
print(end-start,'seconds with',physobj.points.shape[0],'points')

## Vector math prerequisites

In [6]:
@njit
def hamilton_product(p, q, out):
    out0 = p[0]*q[0] - p[1]*q[1] - p[2]*q[2] - p[3]*q[3]
    out1 = p[0]*q[1] + p[1]*q[0] + p[2]*q[3] - p[3]*q[2]
    out2 = p[0]*q[2] - p[1]*q[3] + p[2]*q[0] + p[3]*q[1]
    out3 = p[0]*q[3] + p[1]*q[2] - p[2]*q[1] + p[3]*q[0]
    out[0] = out0
    out[1] = out1
    out[2] = out2
    out[3] = out3

@njit
def quaternion_to_matrix(q):
    s = q[0]
    vx = q[1]
    vy = q[2]
    vz = q[3]
    vx2 = vx*vx
    vy2 = vy*vy
    vz2 = vz*vz
    return np.array([[1-2*vy2-2*vz2, 2*vx*vy-2*s*vz, 2*vx*vz+2*s*vy],
                     [2*vx*vy+2*s*vz, 1-2*vx2-2*vz2, 2*vy*vz-2*s*vx],
                     [2*vx*vz-2*s*vy, 2*vy*vz+2*s*vx, 1-2*vx2-2*vy2]])

@cuda.jit(void(float32[:], float32[:], float32[:]), device=True)
def hamilton_product_device(p, q, out):
    out0 = p[0]*q[0] - p[1]*q[1] - p[2]*q[2] - p[3]*q[3]
    out1 = p[0]*q[1] + p[1]*q[0] + p[2]*q[3] - p[3]*q[2]
    out2 = p[0]*q[2] - p[1]*q[3] + p[2]*q[0] + p[3]*q[1]
    out3 = p[0]*q[3] + p[1]*q[2] - p[2]*q[1] + p[3]*q[0]
    out[0] = out0
    out[1] = out1
    out[2] = out2
    out[3] = out3

@cuda.jit(void(float32[:], float32[:], float32[:]), device=True)
def cross_product_device(p, q, out):
    a0 = p[1] * q[2] - p[2] * q[1]
    a1 = p[2] * q[0] - p[0] * q[2]
    a2 = p[0] * q[1] - p[1] * q[0]
    out[0] = a0
    out[1] = a1
    out[2] = a2

@njit
def cross_product(p, q, out):
    out0 = p[1] * q[2] - p[2] * q[1]
    out1 = p[2] * q[0] - p[0] * q[2]
    out2 = p[0] * q[1] - p[1] * q[0]
    out[0] = out0
    out[1] = out1
    out[2] = out2
    
@cuda.jit(device=True)
def transform(p, pose, out):
    """transform the point p by the pose [qw, qx, qy, qz, tx, ty, tz], where p and out are both represented quaternions with no scalar part"""
    #rotate
    hamilton_product_device(pose, p, out)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    hamilton_product_device(out, pose, out)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    #translate
    out[1] += pose[4]
    out[2] += pose[5]
    out[3] += pose[6]

@njit
def transform_host(p, pose, out):
    """transform the point p by the pose [qw, qx, qy, qz, tx, ty, tz], where p and out are both represented quaternions with no scalar part"""
    #rotate
    hamilton_product(pose, p, out)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    hamilton_product(out, pose, out)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    #translate
    out[1] += pose[4]
    out[2] += pose[5]
    out[3] += pose[6]

## CUDA-based implementation of rigid body simulation

In [7]:
cell_size = 2
    
@cuda.jit
def populate_grid_kernel(points1, pose1, points2, pose2, grid, minPt, gridDim):
    n1 = points1.shape[0]
    n2 = points2.shape[0]
    i = cuda.grid(1)
    transpoint = cuda.local.array(4, float32)
    pose = cuda.local.array(7, float32)
    transpoint[0] = 0
    if i<n1:
        for j in range(3):
            transpoint[j+1] = points1[i,j]
        for j in range(7):
            pose[j] = pose1[j]
        offset = 0
    elif i<n1 + n2:
        i -= n1
        for j in range(3):
            transpoint[j+1] = points2[i,j]
        for j in range(7):
            pose[j] = pose2[j]
        offset = cell_size
    else:
        return
    
    transform(transpoint, pose, transpoint)
    px = int(math.floor((transpoint[1] - minPt[0])/gridDim))
    py = int(math.floor((transpoint[2] - minPt[1])/gridDim))
    pz = int(math.floor((transpoint[3] - minPt[2])/gridDim))
    nx = grid.shape[0]
    ny = grid.shape[1]
    nz = grid.shape[2] // (cell_size*2)
    if px >= 0 and px < nx and py >= 0 and py < ny and pz >= 0 and pz < nz:
        for j in range(cell_size):
            if grid[px, py, pz * 2 * cell_size + offset + j] < 0:
                grid[px, py, pz * 2 * cell_size + offset + j] = i
                break
                
@cuda.jit
def empty_grid(grid):
    i, j, k = cuda.grid(3)
    nx = grid.shape[0]
    ny = grid.shape[1]
    nz = grid.shape[2]
    if i < nx and j < ny and k < nz:
        grid[i, j, k] = -1
                
@cuda.jit
def compute_forces(outForces1, points1, pose1, outForces2, points2, pose2, grid, minPt, gridDim, springK, damping, debug_points1, debug_points2):
    radius = gridDim # maximum distance between interacting particles
    radius2 = radius * radius
    n1 = points1.shape[0]
    n2 = points2.shape[0]
    i = cuda.grid(1)
    transpoint = cuda.local.array(4, float32)
    pose = cuda.local.array(13, float32)
    otherpose = cuda.local.array(13, float32)
    transpoint[0] = 0
    if i<n1:
        for j in range(3):
            transpoint[j+1] = points1[i,j]
        for j in range(13):
            pose[j] = pose1[j]
            otherpose[j] = pose2[j]
        offset = 0
        otherOffset = cell_size
        otherpoints = points2
        outForces = outForces1
        debug_arr = debug_points1
    elif i<n1 + n2:
        i -= n1
        for j in range(3):
            transpoint[j+1] = points2[i,j]
        for j in range(13):
            pose[j] = pose2[j]
            otherpose[j] = pose1[j]
        offset = cell_size
        otherOffset = 0
        otherpoints = points1
        outForces = outForces2
        debug_arr = debug_points2
    else:
        return
    
    transform(transpoint, pose, transpoint)
    #debug
    for k in range(3):
        debug_arr[i,k] = transpoint[k+1]
    point_cm = cuda.local.array(3, float32)
    for k in range(3):
        point_cm[k] = transpoint[k+1] - pose[k+4]
    #print('transpoint:',transpoint[1], transpoint[2], transpoint[3])
    px = int(math.floor((transpoint[1] - minPt[0])/gridDim))
    py = int(math.floor((transpoint[2] - minPt[1])/gridDim))
    pz = int(math.floor((transpoint[3] - minPt[2])/gridDim))
    nx = grid.shape[0]
    ny = grid.shape[1]
    nz = grid.shape[2] // (cell_size*2)
    if px >= 0 and px < nx and py >= 0 and py < ny and pz >= 0 and pz < nz:
        #print('cell:',px, py, pz)
        for j in range(cell_size):
            found = False
            #if grid[px, py, pz * 2 * cell_size + offset + j] >= 0:
                #print('indices:',i, grid[px, py, pz * 2 * cell_size + offset + j])
            if grid[px, py, pz * 2 * cell_size + offset + j] == i:
                found = True
                break
        if not found:
            return
        force = cuda.local.array(3, float32)
        tempforce = cuda.local.array(3, float32)
        torque = cuda.local.array(3, float32)
        temptorque = cuda.local.array(3, float32)
        omega = cuda.local.array(3, float32)
        otheromega = cuda.local.array(3, float32)
        for l in range(3):
            force[l] = 0
            torque[l] = 0
            omega[l] = pose[l+7]
            otheromega[l] = otherpose[l+7]
        
        #print('otheromega:',otheromega[0], otheromega[1], otheromega[2])#, 'othervel:',othervel[0], othervel[1], othervel[2])
            
        vel = cuda.local.array(3, float32)
        #print('velbefore:',vel[0], vel[1], vel[2])
        cross_product_device(omega, point_cm, vel)
        #print('omega:',omega[0], omega[1], omega[2],
        #      'point_cm:',point_cm[0], point_cm[1], point_cm[2],'vel:',vel[0], vel[1], vel[2])
        
        otherpoint = cuda.local.array(4, float32)
        otherpoint_cm = cuda.local.array(3, float32)
        othervel = cuda.local.array(3, float32)
        for dx in range(-1, 2):
            for dy in range(-1, 2):
                for dz in range(-1, 2):
                    for j in range(cell_size):
                        ox = px+dx
                        oy = py+dy
                        oz = pz+dz
                        if ox < 0 or ox >= nx or oy < 0 or oy >= ny or oz < 0 or oz >= nz:
                            continue
                        otherIndex = grid[ox,oy,oz*2*cell_size+otherOffset+j]
                        if otherIndex >= 0:
                            otherpoint[0] = 0
                            for k in range(3):
                                otherpoint[k+1] = otherpoints[otherIndex, k]
                            transform(otherpoint, otherpose, otherpoint)
                            for k in range(3):
                                otherpoint_cm[k] = otherpoint[k+1] - otherpose[k+4]
                                #otherpoint holds the displacement from otherpoint to this point
                                otherpoint[k+1] = transpoint[k+1] - otherpoint[k+1]                                
                            
                            dist2 = otherpoint[1]*otherpoint[1]+otherpoint[2]*otherpoint[2]+otherpoint[3]*otherpoint[3]
                            dist = math.sqrt(dist2)
                            #print(dist)
                            #spring force
                            if dist2 > 0 and dist2 < radius2:
                                    springfac = springK * (radius - dist)/dist
                                    for k in range(3):
                                        tempforce[k] = springfac * otherpoint[k+1]
                                        #print('realvalue:',springfac * otherpoint[k+1],'got:',tempforce[k])
                                        #force[k] += tempforce[k]
                                    cross_product_device(point_cm, tempforce, temptorque)
                                    for k in range(3):
                                        force[k] += tempforce[k]
                                        torque[k] += temptorque[k]
                                        
                            #damping force
                            #compute relative velocity of points
                            cross_product_device(otheromega, otherpoint_cm, othervel)
                            for k in range(3):
                                tempforce[k] = damping * (vel[k] - othervel[k])
                            cross_product_device(point_cm, tempforce, temptorque)
                            for k in range(3):
                                force[k] += tempforce[k]
                                torque[k] += temptorque[k]
                            
        outForces[i, 0] = force[0]
        outForces[i, 1] = force[1]
        outForces[i, 2] = force[2]
        outForces[i, 3] = torque[0]
        outForces[i, 4] = torque[1]
        outForces[i, 5] = torque[2]

TPB = 32


@cuda.jit
def reduce_forces_torques(d_accum, d_f):
    i = cuda.grid(1)
    tIdx = cuda.threadIdx.x
    n = d_f.shape[0]
    sh_w = cuda.shared.array((TPB, 6), dtype=float32)
    #for j in range(6):
    #    sh_w[tIdx, j] = 0
    if i < n:
        for j in range(6):
            sh_w[tIdx, j] = d_f[i, j]
    if i < 6:
        d_accum[i] = 0
    cuda.syncthreads()
    if tIdx < 6:
        component_sum = 0.0
        for j in range(cuda.blockDim.x):
            component_sum += sh_w[j, tIdx]
        cuda.atomic.add(d_accum, tIdx, component_sum)

def simulate_rigid_bodies(v1, f1, p1, m1, v2, f2, p2, m2, gridLen, minPt, maxPt, springK, damping, dt, steps):
    print('minPt:',minPt, 'maxPt:',maxPt, 'gridLen:',gridLen)
    start = time.time()
    objects = [PhysObject(v, f, gridLen, m) for v, f, m in zip((v1, v2), (f1, f2), (m1, m2))]
    end = time.time()
    print('initailized objects in',end-start)
    
    print('points in obj1:',objects[0].points.shape[0])
    print('points in obj2:',objects[1].points.shape[0])
    
    #minPt = np.minimum(np.min(v1, 0), np.min(v2, 0))
    #maxPt = np.maximum(np.max(v1, 0), np.max(v2, 0))
    
    res = np.ceil(((maxPt - minPt) / gridLen)).astype(np.int32)
    res[2] *= cell_size * 2

    TPCB = 4
    blockX = (res[0] + TPCB - 1) // TPCB
    blockY = (res[1] + TPCB - 1) // TPCB
    blockZ = (res[2] + TPCB - 1) // TPCB
    
    
    start = time.time()
    
    grid = cuda.to_device(np.full(res, -1, dtype=np.int32))
    #quaternion as [wxyz], followed by translation [xyz]
    states = []
    states_d = []
    points_d = []
    trans_points_debug_d = [cuda.device_array(obj.points.shape, np.float32) for obj in objects]
    forces_d = [cuda.device_array((obj.points.shape[0], 6), np.float32) for obj in objects] #[force, torque]
    forces_accum_d = [cuda.device_array(6, np.float32) for i in range(2)]
    
    for i in range(2):
        state = np.zeros((13,), dtype=np.float32)
        state[:] = (p1, p2)[i]
        states.append(state)
        #print(state)
        states_d.append(cuda.to_device(state))
        points_d.append(cuda.to_device(objects[i].points))
        
    minPt_d = cuda.to_device(minPt)
    end = time.time()
    print('copied data to GPU in',end-start)
    
    
    m = np.sum([obj.points.shape[0] for obj in objects])
    gridDim = (m + TPB - 1) // TPB
    
    #net conserved state
    L = [st[7:10].copy() for st in states]
    p = [st[10:13].copy() for st in states]
    
    print('initial momentum:',p)
    print('initial angmom:',L)
    
    allstates = []
    
    starte = cuda.event()
    ende = cuda.event()
    
    for i in range(steps):
        print('frame',i)
        starte.record()
        empty_grid[(blockX, blockY, blockZ), (TPCB, TPCB, TPCB)](grid)
        
        #debug
        #gridtest = grid.copy_to_host()
        #if np.sum(gridtest != -1) != 0:
        #    print('NOT PROPERLY CLEARED!', gridtest[gridtest != -1])
        
        populate_grid_kernel[gridDim, TPB](points_d[0], states_d[0], points_d[1], states_d[1], grid, minPt_d, gridLen)
        ende.record()
        ende.synchronize()
        print('populated grid in',cuda.event_elapsed_time(starte, ende)/1000)
        
        starte.record()
        compute_forces[gridDim, TPB](forces_d[0], points_d[0], states_d[0], forces_d[1], points_d[1], states_d[1], grid, minPt_d, gridLen, springK, damping, trans_points_debug_d[0], trans_points_debug_d[1])
        ende.record()
        ende.synchronize()
        print('computed individual point forces in',cuda.event_elapsed_time(starte, ende)/1000)
        
        #debug
        #transpoints = [arr.copy_to_host() for arr in trans_points_debug_d]
        #for j in range(2):
        #    for k in range(objects[j].points.shape[0]):
        #        point = np.zeros(4)
        #        point[1:] = objects[j].points[k,:]
        #        transform_host(point, states[j], point)
        #        diff = transpoints[j][k,:] - point[1:]
        #        print('difference',j,k,':',diff)
        if i == 0:
            gridtest = grid.copy_to_host()
            grid1 = gridtest[:, :, ::(2*cell_size)]
            grid2 = gridtest[:, :, cell_size::(2*cell_size)]
            pl = plot(np.indices(grid1.shape)[:, grid1 >= 0].T, c='blue')
            pl.add_points(np.indices(grid2.shape)[:, grid2 >= 0].T, c='red')
            pl
        #for px in range(res[0]):
        #    for py in range(res[1]):
        #        for pz in range(res[2] // (cell_size*2)):
        #            index1 = gridtest[px, py, pz*2*cell_size]
        #            index2 = gridtest[px, py, pz*2*cell_size+cell_size]
        #            if index1 >= 0 and index2 >= 0:
        #                point1 = np.zeros(4)
        #                point2 = np.zeros(4)
        #                point1[1:] = objects[0].points[index1,:]
        #                point2[1:] = objects[1].points[index2,:]
        #                transform_host(point1, states[0], point1)
        #                transform_host(point2, states[1], point2)
        #                #print('points:',point1, point2)
        #                #point1 += states[0][4:7]
        #                #point2 += states[1][4:7]
        #                diff = point2[1:] - point1[1:]
        #                print('distance',px,py,pz,'between',index1,index2,':',math.sqrt(diff.dot(diff)))
        #end debug
        
        for j in range(2):
            gridDimi = (objects[j].points.shape[0] + TPB - 1) // TPB
            starte.record()
            reduce_forces_torques[gridDimi, TPB](forces_accum_d[j], forces_d[j])
            ende.record()
            ende.synchronize()
            print('reduced forces',j,'on gpu in',cuda.event_elapsed_time(starte, ende)/1000)
            forces_accum = forces_accum_d[j].copy_to_host()
            
            print('force ',j,':',forces_accum[:3])
            print('torque ',j,':',forces_accum[3:])
            
            #debug
            #allforces = forces_d[j].copy_to_host()
            #start = time.time()
            #cpu_force = np.sum(allforces, 0)
            #end = time.time()
            #print('reduced forces',j,'on cpu in',end-start)
            #print('force_gpu:',forces_accum,'force_cpu:',cpu_force)
                        
            p[j] += forces_accum[:3] * dt
            L[j] += forces_accum[3:] * dt
            
            print('momentum ',j,':',p[j])
            print('angular momentum ',j,':',L[j])
            
            #compute auxiliary state
            v = p[j] / objects[j].mass
            
            R = quaternion_to_matrix(states[j][:4])
            Iinv = R @ objects[j].inertia_tensor_inv @ R.T
            omega = Iinv @ L[j]
            
            print('v',j,':',v)
            print('omega',j,':',omega)
                        
            qdot = np.array([0, omega[0], omega[1], omega[2]], np.float32)
            hamilton_product(qdot, states[j][:4], qdot)
            qdot *= 0.5
            
            #timestep update
            states[j][:4] += qdot * dt
            states[j][:4] /= la.norm(states[j][:4])
            states[j][4:7] += v * dt
            states[j][7:10] = omega
            states[j][10:13] = v
            states_d[j].copy_to_device(states[j])
              
        allstates.append([state.copy() for state in states])
        
    return allstates
            

## Testing
Move the slider to advance the time on these examples. You will also see a 3D visualization of the initial grid data structure to help understand the resolution the simulation is running at, with the two objects' occupied grid cells colored red and blue, respectively.

In [8]:
def joinmeshes(v1, f1, v2, f2):
    return np.vstack((v1, v2)), np.vstack((f1, f2 + v1.shape[0]))

In [9]:
def display_animation(v1, f1, v2, f2, states):
    p = plot(v1, f1)
    n = len(states)
    @interact(t=ipywidgets.IntSlider(min=0, max=n-1, step=1))
    def ff(t):
        vs = []
        fs = []
        for j in range(2):
            v = (v1, v2)[j]
            f = (f1, f2)[j]
            R = quaternion_to_matrix(states[t][j][:4])
            x = states[t][j][4:7]
            newv = (R @ v.T).T + x
            vs.append(newv)
            fs.append(f)
        fullv, fullf = joinmeshes(vs[0], fs[0], vs[1], fs[1])
        plot(fullv, fullf, plot=p)
    p

### off-center collision

In [10]:
v, f = igl.read_triangle_mesh('data/cylinder.obj')
pose1 = np.array([1, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0], np.float32)
pose2 = np.array([math.sqrt(2)/2, 0, math.sqrt(2)/2, 0, 0.5, 2.1, 0,
                 0, 0, 0, 0, -10, 0], np.float32)
states = simulate_rigid_bodies(v, f, pose1, 1, v, f, pose2, 1, 0.02, np.array([-1.1, -1.1, -1.1]), np.array([1.1, 3.1, 1.1]), 100, 0, 0.001, 1000)
display_animation(v, f, v, f, states)

minPt: [-1.1 -1.1 -1.1] maxPt: [1.1 3.1 1.1] gridLen: 0.02
initailized objects in 1.8911149501800537
points in obj1: 781953
points in obj2: 781953
copied data to GPU in 0.11768770217895508
initial momentum: [array([0., 0., 0.], dtype=float32), array([  0., -10.,   0.], dtype=float32)]
initial angmom: [array([0., 0., 0.], dtype=float32), array([0., 0., 0.], dtype=float32)]
frame 0
populated grid in 0.43843368530273436
computed individual point forces in 0.003115040063858032


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(55.5, 55.…

reduced forces 0 on gpu in 0.0009361600279808044
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0005895040035247803
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985616  0.       ]
omega 1 : [0. 0. 0.]
frame 1
populated grid in 0.0015269440412521362
computed individual point forces in 0.0029143359661102297
reduced forces 0 on gpu in 0.0005868160128593444
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0005872640013694764
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985616  0.       ]
omega 1 : [0. 0. 0.]
frame 2
populated grid in 0.0019964799880981443
computed individual point 

reduced forces 0 on gpu in 0.000587776005268097
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0005873919725418091
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985616  0.       ]
omega 1 : [0. 0. 0.]
frame 19
populated grid in 0.0015215040445327758
computed individual point forces in 0.005509119987487793
reduced forces 0 on gpu in 0.0005876479744911194
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0005869119763374328
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985616  0.       ]
omega 1 : [0. 0. 0.]
frame 20
populated grid in 0.0015316799879074097
computed individual point 

reduced forces 0 on gpu in 0.00044115200638771056
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004406079947948456
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985616  0.       ]
omega 1 : [0. 0. 0.]
frame 38
populated grid in 0.00138428795337677
computed individual point forces in 0.0021731200218200685
reduced forces 0 on gpu in 0.0004408000111579895
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004405120015144348
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985616  0.       ]
omega 1 : [0. 0. 0.]
frame 39
populated grid in 0.0013721599578857423
computed individual point

...

populated grid in 0.0013168959617614747
computed individual point forces in 0.0017405439615249634
reduced forces 0 on gpu in 0.00044223999977111814
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.650236   -7.489552   -0.15641017]
angular momentum  0 : [-0.10426518  0.07085888  1.3503418 ]
v 0 : [-0.42365652 -1.1972511  -0.02500313]
omega 0 : [-0.16438124  0.14777578  2.45948981]
reduced forces 1 on gpu in 0.0004423680007457733
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.763116   -2.4638066   0.12588638]
angular momentum  1 : [-0.17257762  0.02018274  0.12736084]
v 1 : [ 0.44170108 -0.39385468  0.02012371]
omega 1 : [-0.33350784  0.03922955  0.18745617]
frame 951
populated grid in 0.0018405120372772218
computed individual point forces in 0.0030420479774475096
reduced forces 0 on gpu in 0.0004424639940261841
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.650236   -7.489552   -0.15641017]
angular momentum  0 : [-0.10426518  0.07085888  1.3

computed individual point forces in 0.0017214080095291138
reduced forces 0 on gpu in 0.00044281598925590516
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.650236   -7.489552   -0.15641017]
angular momentum  0 : [-0.10426518  0.07085888  1.3503418 ]
v 0 : [-0.42365652 -1.1972511  -0.02500313]
omega 0 : [-0.16533572  0.14903938  2.45934985]
reduced forces 1 on gpu in 0.0004423680007457733
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.763116   -2.4638066   0.12588638]
angular momentum  1 : [-0.17257762  0.02018274  0.12736084]
v 1 : [ 0.44170108 -0.39385468  0.02012371]
omega 1 : [-0.33354281  0.03882103  0.18747367]
frame 973
populated grid in 0.0017873920202255249
computed individual point forces in 0.0017305279970169067
reduced forces 0 on gpu in 0.0004423680007457733
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.650236   -7.489552   -0.15641017]
angular momentum  0 : [-0.10426518  0.07085888  1.3503418 ]
v 0 : [-0.42365652 -1.1972511  

frame 995
populated grid in 0.0017671680450439453
computed individual point forces in 0.0017135039567947388
reduced forces 0 on gpu in 0.0004433920085430145
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.650236   -7.489552   -0.15641017]
angular momentum  0 : [-0.10426518  0.07085888  1.3503418 ]
v 0 : [-0.42365652 -1.1972511  -0.02500313]
omega 0 : [-0.16640141  0.15030742  2.45920108]
reduced forces 1 on gpu in 0.0006225919723510743
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.763116   -2.4638066   0.12588638]
angular momentum  1 : [-0.17257762  0.02018274  0.12736084]
v 1 : [ 0.44170108 -0.39385468  0.02012371]
omega 1 : [-0.33357699  0.03839406  0.18749499]
frame 996
populated grid in 0.001312608003616333
computed individual point forces in 0.0017074559926986694
reduced forces 0 on gpu in 0.0004423680007457733
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.650236   -7.489552   -0.15641017]
angular momentum  0 : [-0.10426518  0.07085

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

interactive(children=(IntSlider(value=0, description='t', max=999), Output()), _dom_classes=('widget-interact'…

### rotating in place

In [11]:
v, f = igl.read_triangle_mesh('data/cylinder.obj')
pose1 = np.array([1, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0], np.float32)
pose2 = np.array([1, 0, 0, 0, 0, 2.1, 0,
                 0, 10, 0, 0, 0, 0], np.float32)
states = simulate_rigid_bodies(v, f, pose1, 1, v, f, pose2, 1, 0.02, np.array([-1.1, -1.1, -1.1]), np.array([1.1, 3.1, 1.1]), 100, 0, 0.001, 250)
display_animation(v, f, v, f, states)

minPt: [-1.1 -1.1 -1.1] maxPt: [1.1 3.1 1.1] gridLen: 0.02
initailized objects in 1.0083017349243164
points in obj1: 781953
points in obj2: 781953
copied data to GPU in 0.019946575164794922
initial momentum: [array([0., 0., 0.], dtype=float32), array([0., 0., 0.], dtype=float32)]
initial angmom: [array([0., 0., 0.], dtype=float32), array([ 0., 10.,  0.], dtype=float32)]
frame 0
populated grid in 0.0017246400117874145
computed individual point forces in 0.0014121600389480592


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(55.5, 55.…

reduced forces 0 on gpu in 0.00044441598653793336
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004435200095176697
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 1.73997148e-05  8.21019650e+00 -5.79459254e-06]
frame 1
populated grid in 0.00128928005695343
computed individual point forces in 0.0014258240461349487
reduced forces 0 on gpu in 0.0004427199959754944
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00044249600172042845
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 1.73544740e-05  8.21019650e+00 -5.96390169e-06]
frame 2
populated grid in 0.002142911911010742
com

populated grid in 0.0012889599800109863
computed individual point forces in 0.0015617280006408692
reduced forces 0 on gpu in 0.0004419839978218079
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00067740797996521
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 1.63442279e-05  8.21019650e+00 -8.77693436e-06]
frame 19
populated grid in 0.0012915199995040894
computed individual point forces in 0.0015635839700698852
reduced forces 0 on gpu in 0.00044268798828125
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004423680007457733
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 1.6

reduced forces 0 on gpu in 0.0006733440160751343
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00044281598925590516
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 1.47940897e-05  8.21019650e+00 -1.15672678e-05]
frame 37
populated grid in 0.00134553599357605
computed individual point forces in 0.0016568319797515868
reduced forces 0 on gpu in 0.0004432959854602814
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004423680007457733
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 1.46940984e-05  8.21019650e+00 -1.17150263e-05]
frame 38
populated grid in 0.0013301440477371217
c

...

reduced forces 0 on gpu in 0.0004443199932575226
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004423680007457733
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [-1.16551620e-05  8.21019650e+00 -1.79481422e-05]
frame 196
populated grid in 0.0014306240081787109
computed individual point forces in 0.0026117119789123536
reduced forces 0 on gpu in 0.0004427199959754944
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00044275200366973875
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [-1.18303566e-05  8.21019650e+00 -1.78568204e-05]
frame 197
populated grid in 0.00190611195564270

populated grid in 0.0014310719966888427
computed individual point forces in 0.0025432639122009277
reduced forces 0 on gpu in 0.0004423680007457733
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004421760141849518
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [-1.51661770e-05  8.21019650e+00 -1.56912797e-05]
frame 217
populated grid in 0.0014292160272598266
computed individual point forces in 0.003899616003036499
reduced forces 0 on gpu in 0.0006363199949264526
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004442560076713562
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [

angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00044249600172042845
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [-1.81026924e-05  8.21019650e+00 -1.29196663e-05]
frame 237
populated grid in 0.0014143359661102294
computed individual point forces in 0.00395251202583313
reduced forces 0 on gpu in 0.0004442879855632782
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004434880018234253
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [-1.82370546e-05  8.21019650e+00 -1.27664107e-05]
frame 238
populated grid in 0.0014162240028381347
computed individual point forces in 0.0024467198848724366
reduced forces 0 on gpu in 0.000442656010389328
force  0 : [

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

interactive(children=(IntSlider(value=0, description='t', max=249), Output()), _dom_classes=('widget-interact'…

### Interlocking toruses

In [12]:
v, f = igl.read_triangle_mesh('data/torus.obj')
pose1 = np.array([math.sqrt(2)/2, 0, 0, math.sqrt(2)/2, 0, 0, 0,
                 0, 0, 0, 0, 0, 0], np.float32)
pose2 = np.array([math.sqrt(2)/2, math.sqrt(2)/2, 0, 0, 0, 1, 0,
                 0, 1, 0, 0, 0, 0], np.float32)
states = simulate_rigid_bodies(v, f, pose1, 1, v, f, pose2, 1, 0.04, np.array([-2.1, -2.1, -2.1]), np.array([2.1, 3.1, 2.1]), 100, 0, 0.001, 1000)
display_animation(v, f, v, f, states)

minPt: [-2.1 -2.1 -2.1] maxPt: [2.1 3.1 2.1] gridLen: 0.04
initailized objects in 0.04787087440490723
points in obj1: 18294
points in obj2: 18294
copied data to GPU in 0.00997304916381836
initial momentum: [array([0., 0., 0.], dtype=float32), array([0., 0., 0.], dtype=float32)]
initial angmom: [array([0., 0., 0.], dtype=float32), array([0., 1., 0.], dtype=float32)]
frame 0
populated grid in 0.0001908160001039505
computed individual point forces in 7.769600301980973e-05


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(51.5, 52.…

reduced forces 0 on gpu in 1.651199907064438e-05
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 1.756799966096878e-05
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [ 2.70532901e-03  2.49471228e+01 -4.37405096e-04]
frame 1
populated grid in 0.00019046400487422944
computed individual point forces in 7.577600330114365e-05
reduced forces 0 on gpu in 1.635199971497059e-05
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 1.4336000196635724e-05
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [ 2.70271509e-03  2.49471295e+01 -4.44926329e-04]
frame 2
populated grid in 0.0001904959976673126
comput

reduced forces 1 on gpu in 1.5552000142633914e-05
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [ 2.64394771e-03  2.49471240e+01 -8.02926917e-04]
frame 34
populated grid in 0.00018982400000095367
computed individual point forces in 7.555200159549713e-05
reduced forces 0 on gpu in 1.5168000012636184e-05
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 1.4336000196635724e-05
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [ 2.64211244e-03  2.49471334e+01 -8.10080625e-04]
frame 35
populated grid in 0.00019046400487422944
computed individual point forces in 8.828800171613694e-05
reduced forces 0 on gpu in 1.8208000808954238e-05
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momen

populated grid in 0.00019251200556755067
computed individual point forces in 7.990399748086929e-05
reduced forces 0 on gpu in 1.756799966096878e-05
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-0.01341514 -0.01148621 -0.04340385]
angular momentum  0 : [-0.03610816  1.413284    0.00349682]
v 0 : [-0.01145794 -0.00981043 -0.03707146]
omega 0 : [-1.07728758e-01  1.28534690e+01 -4.34718135e-03]
reduced forces 1 on gpu in 1.6383999958634378e-05
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [-0.01912172 -0.00661491  0.03769523]
angular momentum  1 : [-0.00731455 -0.48662215 -0.00790924]
v 1 : [-0.01633196 -0.00564983  0.03219569]
omega 1 : [ -0.03303914 -12.13970645  -0.02474088]
frame 63
populated grid in 0.0001912959963083267
computed individual point forces in 7.887999713420867e-05
reduced forces 0 on gpu in 1.6095999628305435e-05
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-0.01341514 -0.01148621 -0.04340385]
angular momentum  0 : [-0.03610816 

...

populated grid in 0.0002457599937915802
computed individual point forces in 0.0001409280002117157
reduced forces 0 on gpu in 2.0128000527620317e-05
force  0 : [ 28.4549    14.317428 -79.101   ]
torque  0 : [-68.37978     -0.42269015 -27.131832  ]
momentum  0 : [-0.60902315  1.5533978   0.92813855]
angular momentum  0 : [0.67350113 0.606557   0.61461335]
v 0 : [-0.52016985  1.3267652   0.79272795]
omega 0 : [3.50901058 2.93238163 1.96723902]
reduced forces 1 on gpu in 2.0479999482631683e-05
force  1 : [-20.509111 -13.486715  71.93806 ]
torque  1 : [ 8.679759 68.40035  16.588501]
momentum  1 : [ 0.49975657 -1.3697375  -0.95548105]
angular momentum  1 : [-0.0610059  -0.62507606  1.3268864 ]
v 1 : [ 0.4268447  -1.1699     -0.81608135]
omega 1 : [-4.99086433 -7.41622932  4.48267159]
frame 934
populated grid in 0.0002443840056657791
computed individual point forces in 0.0001368959993124008
reduced forces 0 on gpu in 1.9039999693632127e-05
force  0 : [ 33.23999   20.337963 -76.636055]
torque 

force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 0.13591164 -1.5691453   0.39862633]
angular momentum  1 : [0.15967034 0.65072036 1.5993747 ]
v 1 : [ 0.11608284 -1.3402152   0.34046882]
omega 1 : [ 8.554882   15.95329344  8.34806913]
frame 962
populated grid in 0.00024428799748420714
computed individual point forces in 0.00010527999699115753
reduced forces 0 on gpu in 1.9840000197291375e-05
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-0.19050434  1.7385395  -0.47928447]
angular momentum  0 : [-0.59616566  0.64091235  0.20787762]
v 0 : [-0.16271076  1.4848956  -0.40935937]
omega 0 : [-3.77033604  5.20677833 -0.9582635 ]
reduced forces 1 on gpu in 1.852799952030182e-05
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 0.13591164 -1.5691453   0.39862633]
angular momentum  1 : [0.15967034 0.65072036 1.5993747 ]
v 1 : [ 0.11608284 -1.3402152   0.34046882]
omega 1 : [ 8.42373891 15.98496267  8.35016672]
frame 963
populated grid in 0.00024592000246047974

frame 995
populated grid in 0.0002443840056657791
computed individual point forces in 0.00014697599411010742
reduced forces 0 on gpu in 2.0031999796628952e-05
force  0 : [ 79.57577  -14.30999   45.909534]
torque  0 : [-10.683811 -24.86682   19.23983 ]
momentum  0 : [ 0.44761196  1.6510829  -0.02671104]
angular momentum  0 : [-0.7053591   0.36375114  0.3588206 ]
v 0 : [ 0.3823077   1.4101985  -0.02281404]
omega 0 : [-4.14134979  2.74643408  0.19187274]
reduced forces 1 on gpu in 1.8912000581622124e-05
force  1 : [-79.76746   23.92227  -48.164913]
torque  1 : [ 38.40487  -20.966446 -82.54104 ]
momentum  1 : [-0.5063767  -1.4518088  -0.07182547]
angular momentum  1 : [0.5545291  0.46536684 0.9616248 ]
v 1 : [-0.43249896 -1.2399975  -0.0613465 ]
omega 1 : [ 6.82852607 14.82037916  5.13547805]
frame 996
populated grid in 0.00024451200664043425
computed individual point forces in 0.00014870400726795196
reduced forces 0 on gpu in 2.0479999482631683e-05
force  0 : [ 73.6503    -11.5055485  40.

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

interactive(children=(IntSlider(value=0, description='t', max=999), Output()), _dom_classes=('widget-interact'…

## Conclusions
Despite there being on the order of millions of points per object, this method can simulate a 1000 timestep simulation in just a couple of seconds on my RTX 2060S.

There are some drawbacks to the scatter-based approach to storing particles in the grid. Namely, there can potentially be race conditions such that some particles do not get recorded (due to being overwritten). A thread cannot know where there is a free space to store the particle index in the cell, so it scans for an empty space, leading to this problem. However, I have partitioned the cells into different regions for each object so that interactions between the objects are not missed as long as each has a particle in the vicinity. Furthermore, I only compute forces on particles that are actually in the data structure, because otherwise it would be possible for more particles to receive forces from the other object than were detected as neighbors in the other direction, breaking Newton's third law.